In [13]:
import os, json, re, datetime as dt
import pandas as pd, numpy as np
import plotly.express as px
import ipywidgets as w
import plotly.graph_objects as go
from IPython.display import display, Markdown
from ipyfilechooser import FileChooser
from ipywidgets import DatePicker
from IPython.display import Image, Markdown
from statsmodels.formula.api import ols
import statsmodels.api as sm
from IPython.display import FileLink
import warnings
import io
#import xlsxwriter

In [14]:
PHASE_RE = re.compile(r"\s+(start|end)\s*$", flags=re.I)

def split_op(name: str):
    """Extract base + phase (_start/_end) from feature name."""
    name = name.strip()
    if name.lower().endswith("_start"):
        return name[:-6], "start"
    elif name.lower().endswith("_end"):
        return name[:-4], "end"
    return name, None

def canonical_op(name: str):
    return re.sub(r"\s+(start|end)\s*$", "", name, flags=re.I).strip()

def cp_calc(series, tol):
    σ = series.std(ddof=0)
    return np.nan if σ == 0 or pd.isna(tol) or tol == 0 else (2 * tol) / (6 * σ)

def find_closest_image(report_path: str, time_hint: dt.datetime) -> str:
    img_folder = os.path.dirname(report_path).replace("reports", "images")
    if not os.path.exists(img_folder):
        return None

    min_diff = dt.timedelta(seconds=30)  # threshold to consider
    closest_img = None

    for fname in os.listdir(img_folder):
        if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        try:
            # Extract timestamp from image name: Task imageYYYY-MM-DD HH-MM-SS-mmm.png
            ts_part = fname.replace("Task image", "").replace(".png", "")
            ts_part = ts_part.replace("-", " ", 1)  # split once after date
            ts_dt = dt.datetime.strptime(ts_part, "%Y-%m-%d %H-%M-%S-%f")

            if abs(ts_dt - time_hint) < min_diff:
                min_diff = abs(ts_dt - time_hint)
                closest_img = os.path.join(img_folder, fname)
        except Exception:
            continue

    return closest_img

def load_skillreal_folder(folder: str) -> pd.DataFrame:
    """Walk a folder, read every *.json inside 'reports/' subfolders, attach first image from sibling 'images/', return flattened DataFrame."""
    records = []

    for root, _, files in os.walk(folder):
        if "reports" not in os.path.basename(root).lower():
            continue  # only read from folders named 'reports'

        for fn in files:
            if not fn.lower().endswith(".json"):
                continue

            fpath = os.path.join(root, fn)
            try:
                with open(fpath, "r") as fh:
                    data = json.load(fh)

                # Get first image in sibling 'images' folder
                images_folder = os.path.dirname(fpath).replace("Reports", "images")
                image_path = None
                if os.path.exists(images_folder):
                    for img_fn in sorted(os.listdir(images_folder)):
                        if img_fn.lower().endswith((".png", ".jpg", ".jpeg")):
                            image_path = os.path.join(images_folder, img_fn)
                            break

                for op in data.get("operationsData", []):
                    op_name = op.get("operationName", "").strip()
                    model_class = op.get("aiModelclass")
                    porosity = op.get("qualityClassificationResult", {}).get("porosityClassificationResult", {})
                    max_pore_diameter = porosity.get("maxPoreDiameterFound")
                    total_pores_diameter = porosity.get("totalPoresDiameter")
                    pore_diameters = porosity.get("poreDiameters", [])
                    num_pores = len(pore_diameters)

                    for c in op.get("coordianteList", []):
                        feat_name = c.get("name", "").strip()
                        op_base, op_phase = split_op(feat_name)

                        records.append({
                            "op_name": op_name,
                            "op_base": op_base,
                            "op_phase": op_phase,
                            "op_variant": f"{op_base} ({op_phase})" if op_phase else op_base,
                            "model_class": model_class,
                            "feature": feat_name,
                            "dev_x": c.get("deviation", {}).get("x"),
                            "dev_y": c.get("deviation", {}).get("y"),
                            "dev_z": c.get("deviation", {}).get("z"),
                            "deviationSize": c.get("deviationSize"),
                            "tolerance": c.get("tolerance"),
                            "timestamp": dt.datetime.fromtimestamp(os.path.getmtime(fpath)),
                            "file": os.path.abspath(fpath),
                            "status": c.get("status"),
                            "phy_x": c.get("physicalLocation", {}).get("x"),
                            "phy_y": c.get("physicalLocation", {}).get("y"),
                            "phy_z": c.get("physicalLocation", {}).get("z"),
                            "maxPoreDiameterFound": max_pore_diameter,
                            "totalPoresDiameter": total_pores_diameter,
                            "numPores" : num_pores,
                            "image_path": image_path
                        })

            except (json.JSONDecodeError, UnicodeDecodeError) as exc:
                print(f"[WARN] {fpath}: {exc}")
                continue

    return pd.DataFrame.from_records(records)

In [15]:
def compute_feature_lengths(df: pd.DataFrame) -> pd.DataFrame:
    """Find _start/_end coordinate pairs and compute Euclidean length."""
    grouped = df[df["op_phase"].isin(["start", "end"])][
        ["op_base", "op_phase", "phy_x", "phy_y", "phy_z", "file", "op_name", "timestamp"]
    ].dropna().copy()

    results = []
    for (op_base, file), group in grouped.groupby(["op_base", "file"], sort=False):
        starts = group[group["op_phase"] == "start"]
        ends   = group[group["op_phase"] == "end"]

        if len(starts) == 1 and len(ends) == 1:
            start, end = starts.iloc[0], ends.iloc[0]
            length = np.sqrt(
                (end["phy_x"] - start["phy_x"])**2 +
                (end["phy_y"] - start["phy_y"])**2 +
                (end["phy_z"] - start["phy_z"])**2
            )
            results.append({
                "op_base": op_base,
                "length" : length,
                "file"   : file,
                "op_name": start["op_name"],
                "timestamp": start["timestamp"]
            })
    return pd.DataFrame(results)

In [16]:
master_df   = pd.DataFrame()
length_df   = pd.DataFrame()
loaded_files = set()

In [17]:
folder_chooser = FileChooser(os.getcwd(), title='Select Folder with JSON files')
folder_chooser.use_dir_icons = True
folder_chooser.show_only_dirs = True

load_folder_btn = w.Button(description="Load Folder", button_style="success")
reset_btn       = w.Button(description="Reset",       button_style="danger")
save_button = w.Button(description="Save to Excel",button_style="success",tooltip="Click to save formatted Excel file", icon="save")

op_dd    = w.Dropdown(description="Operation")
axis_dd  = w.Dropdown(description="Metric",
                      options=[("Deviation size", "deviationSize"),
                               ("Dev-X", "dev_x"), ("Dev-Y", "dev_y"), ("Dev-Z", "dev_z")])
scale_dd = w.Dropdown(description="Y-scale",
                      options=[("Linear", "linear"), ("Log", "log")])
view_dd  = w.Dropdown(description="View",
                      options=[("Skillreal Co-ordinate", "coordinates"),
                               ("Vehicle Co-ordinate", "deviation")],
                      value="coordinates")
metric_dd = w.Dropdown(description="metric",
                       options=[("Records", "records"), ("Timestamps", "timestamps")],
                       value="records")
gauge_btn = w.Button(description="Compute Gauge R&R", button_style="info")
start_date_picker = DatePicker(
    description='Start Date',
    disabled=False
)
end_date_picker = DatePicker(
    description='End Date',
    disabled=False
)

# output areas
plot_out, plot_total_out, plot_y_out, plot_x_out          = w.Output(), w.Output(), w.Output(), w.Output()
plot_y_vehicle_out, plot_z_vehicle_out                    = w.Output(), w.Output()
plot_status                                               = w.Output()
plot_len_out, plot_max_poro_out, plot_total_poro_out      = w.Output(), w.Output(), w.Output()
stats_out                                                 = w.Output()
plot_pore_count_out                                       = w.Output()
image_out                                                 = w.Output()
gauge_out                                                 = w.Output()


In [18]:
def attach_image_click(fig_widget, sub_df):
    def on_click(trace, points, selector):
        if points.point_inds and points.trace_index is not None:
            point_index = points.point_inds[0]
            trace_obj = fig_widget.data[points.trace_index]

            if "customdata" in trace_obj and point_index < len(trace_obj.customdata):
                row_idx = trace_obj.customdata[point_index][0]
                if row_idx in sub_df.index:
                    row = sub_df.loc[row_idx]
                    img_path = row.get("image_path")
                    with image_out:
                        image_out.clear_output()
                        if img_path and os.path.exists(img_path):
                            display(Image(filename=img_path))
                        else:
                            display(Markdown("**Image not found for selected point.**"))

    for trace in fig_widget.data:
        trace.on_click(on_click)

    return fig_widget

In [19]:
def create_scatter_trace(x, y, x_label, y_label, sub, color="steelblue"):
    return go.Scatter(
        x=x,
        y=y,
        mode="markers",
        marker=dict(size=6, color=color),
        customdata=sub[["row_idx"]],
        name="",
        hovertemplate=f"{x_label}=%{{x}}<br>{y_label}=%{{y}}<extra></extra>"
    )

In [20]:
def count_domain_outliers(df, rules):
    counts = {}
    for col, condition in rules.items():
        if col in df.columns:
            counts[col] = df[col].apply(condition).sum()
    return counts

In [ ]:
def update_dashboard(_):
    if master_df.empty:
        return

    # ------------ filter & prep ------------
    sub = master_df[master_df["feature"] == op_dd.value].copy()
    sub["timestamp"] = pd.to_datetime(sub["timestamp"], errors='coerce')
    start_date = start_date_picker.value
    end_date   = end_date_picker.value
    if start_date and end_date:
        sub = sub[(sub["timestamp"].dt.date >= start_date) & (sub["timestamp"].dt.date <= end_date)]
    sub = sub.sort_values(["timestamp", "file"])
    sub["rec_no"] = np.arange(1, len(sub) + 1)
    sub = sub.copy()
    sub["rec_no"] = np.arange(1, len(sub) + 1)
    sub["row_idx"] = sub.index  # ← this line is key
    metric = axis_dd.value
    mc = ", ".join(sub["model_class"].dropna().unique()) or "n/a"
    counts = sub['status'].value_counts().reset_index()
    counts.columns = ['status', 'count']
    x_label = "Record Number" if metric_dd.value == "records" else "timestamp"
    rule_max = {"totalPoresDiameter": lambda x: x>6.00}
    rule_sum = {"maxPoreDiameterFound": lambda x: x>1.6}
    max_outlier_counts = count_domain_outliers(sub, rule_max)
    sum_outlier_counts = count_domain_outliers(sub, rule_sum)
    print(max_outlier_counts)
    print(sum_outlier_counts)
    
    #-------------Plot 1: Total Deviation---------------
    with plot_total_out:
        plot_total_out.clear_output()

        sub["_file_short"] = sub["file"].apply(lambda p: os.path.basename(p))
        trace_total = create_scatter_trace(
        x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
        y=sub["deviationSize"],
        x_label=x_label,
        y_label="Total Deviation",
        sub=sub
    )

        fig_widget_total = go.FigureWidget(data=[trace_total])
        fig_widget_total.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Total Deviation", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Total Deviation (mm)",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        attach_image_click(fig_widget_total, sub)
        display(fig_widget_total)

    #---------------------Plot 2: Deviation X-----------------
    with plot_x_out:
        plot_x_out.clear_output()

        trace_x = create_scatter_trace(
        x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
        y=sub["dev_x"],
        x_label="Record Number" if metric_dd.value == "records" else "timestamp",
        y_label="dev_x",
        sub=sub
    )

        fig_widget_x = go.FigureWidget(data=[trace_x])
        fig_widget_x.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Deviation X", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Deviation X (mm)",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        attach_image_click(fig_widget_x, sub)
        display(fig_widget_x)

    #----------------Plot 3: Deviation Y----------------------
    with plot_y_out:
        plot_y_out.clear_output()

        trace_y = create_scatter_trace(
            x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
            y=sub["dev_y"],
            x_label="rec_no" if metric_dd.value == "records" else "timestamp",
            y_label="dev_y",
            sub=sub
        )

        fig_widget_y = go.FigureWidget(data=[trace_y])
        fig_widget_y.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Deviation Y", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Deviation Y (mm)",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        attach_image_click(fig_widget_y, sub)
        display(fig_widget_y)

    #---------------------Plot 4: Deviation Z-------------------------
    with plot_out:
        plot_out.clear_output()

        trace_z = create_scatter_trace(
            x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
            y=sub["dev_z"],
            x_label="Record Number" if metric_dd.value == "records" else "timestamp",
            y_label="dev_z",
            sub=sub
        )

        fig_widget_z = go.FigureWidget(data=[trace_z])
        fig_widget_z.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Deviation z", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Deviation Z (mm)",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        attach_image_click(fig_widget_z, sub)
        display(fig_widget_z)

    #-----------------------Plot 5/6: Vehicle Frame-----------------------------
    with plot_y_vehicle_out:
        plot_y_vehicle_out.clear_output()

        trace_vehicle_y = create_scatter_trace(
            x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
            y=sub["dev_z"],
            x_label="rec_no" if metric_dd.value == "records" else "timestamp",
            y_label="dev_y",
            sub=sub
        )

        fig_widget_vehicle_y = go.FigureWidget(data=[trace_vehicle_y])
        fig_widget_vehicle_y.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Deviation Y", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Deviation Y (mm)",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        attach_image_click(fig_widget_vehicle_y, sub)
        display(fig_widget_vehicle_y)

    with plot_z_vehicle_out:
        plot_z_vehicle_out.clear_output()

        trace_vehicle_z = create_scatter_trace(
            x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
            y=sub["dev_y"],
            x_label="rec_no" if metric_dd.value == "records" else "timestamp",
            y_label="dev_z",
            sub=sub
        )

        fig_widget_vehicle_z = go.FigureWidget(data=[trace_vehicle_z])
        fig_widget_vehicle_z.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Deviation z", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Deviation Z (mm)",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        attach_image_click(fig_widget_vehicle_z, sub)
        display(fig_widget_vehicle_z)

    #---------------Plot 7: Status-----------------
    with plot_status:
        plot_status.clear_output()
        # Group by feature and compute inspection rate
        status_group = (
            master_df.groupby('feature')
            .agg(inspected=('status', lambda x: ((x == 1) | (x == 5)).sum()),
                 total=('status', 'count'))
            .reset_index()
        )
        status_group['inspection_rate'] = (status_group['inspected'] / status_group['total']) * 100

        fig7 = px.scatter(
            status_group, x=range(len(status_group)), y='inspection_rate',
            hover_name='feature',
            title="Inspection Rate of Phase 1 Features - LH Sample (Status = 1(Ok) or Status = 5(Misplaced))",
            labels={"x": "Feature Index", "inspection_rate": "Inspection Rate (%)"}
        )
        fig7.update_traces(marker=dict(size=6, color="steelblue"))  # consistent color & size
        fig7.update_layout(
            title_x=0.5,
            xaxis_title="Record Number",
            yaxis_title="Percent Rate",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
        )
        fig7.update_xaxes(showgrid=False, showline=True, linecolor="black", tickmode="linear", dtick=50)
        fig7.update_yaxes(showgrid=False, showline=True, linecolor="black", range=[-10,110], dtick=10)
        display(fig7)

    #--------------------PLot 8: Mig Length -----------------
    with plot_len_out:
        plot_len_out.clear_output()

        # Convert the selected feature name into its base (removing _start/_end)
        selected_base, _ = split_op(op_dd.value)

        # Filter for all MIG lengths associated with this base
        length_subset = length_df[length_df["op_base"] == selected_base]

        if length_subset.empty:
            display(Markdown("**No MIG length records available for this feature.**"))
        else:
            length_subset = length_subset.copy()
            length_subset["row_idx"] = length_subset.index

            trace_mig_len = go.Scatter(
                x=length_subset.index if metric_dd.value == "records" else length_subset["timestamp"].astype("O"),
                y=length_subset["length"],
                mode="markers",
                marker=dict(size=6, color="steelblue"),
                customdata=length_subset[["row_idx"]],
                hovertemplate=f"{x_label}=%{{x}}<br>Length=%{{y:.2f}} mm<extra></extra>",
                name=""
            )

            fig_widget_mig_len = go.FigureWidget(data=[trace_mig_len])
            fig_widget_mig_len.update_layout(
                title=dict(text=f"Mig Length for {mc} {op_dd.value}", x=0.5),
                xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
                yaxis_title="Mig Length (mm)",
                plot_bgcolor="white",
                paper_bgcolor="white",
                margin=dict(l=60, r=60, t=50, b=50),
                showlegend=False,
                xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
                yaxis=dict(showgrid=False, showline=True, linecolor="black"),
                height=500,
                width=1100,
            )

            attach_image_click(fig_widget_mig_len, length_subset)
            display(fig_widget_mig_len)

    #--------------Plot 9/10: Porosity------------------
    with plot_max_poro_out:
        plot_max_poro_out.clear_output()
        trace_max_poro = go.Scatter(
            x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
            y=sub["maxPoreDiameterFound"],
            mode="markers",
            marker=dict(size=6, color="steelblue"),
            customdata=sub[["row_idx"]],
            hovertemplate=f"{x_label}=%{{x}}<br>Max Pore=%{{y:.2f}} mm<extra></extra>",
            name=""
        )

        fig_widget_max_poro = go.FigureWidget(data=[trace_max_poro])
        fig_widget_max_poro.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Max Porosity Diameter", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Max Pores (mm)",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        outlier_threshold = 1.6
        fig_widget_max_poro.add_shape(
            type="line",
            x0=sub["rec_no"].min() if metric_dd.value == "records" else sub["timestamp"].min(),
            x1=sub["rec_no"].max() if metric_dd.value == "records" else sub["timestamp"].max(),
            y0=outlier_threshold,
            y1=outlier_threshold,
            line=dict(
                color="red",
                width=2,
                dash="dot"  # 'dot' for dotted line
            )
        )

        attach_image_click(fig_widget_max_poro, sub)
        display(fig_widget_max_poro)

    with plot_total_poro_out:
        plot_total_poro_out.clear_output()
        trace_total_poro = go.Scatter(
            x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
            y=sub["totalPoresDiameter"],
            mode="markers",
            marker=dict(size=6, color="steelblue"),
            customdata=sub[["row_idx"]],
            hovertemplate=f"{x_label}=%{{x}}<br>Total Pores=%{{y:.2f}} mm<extra></extra>",
            name=""
        )

        fig_widget_total_poro = go.FigureWidget(data=[trace_total_poro])
        fig_widget_total_poro.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Total Pores Diameter", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Total Pores (mm)",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        outlier_threshold = 6.0
        fig_widget_total_poro.add_shape(
            type="line",
            x0=sub["rec_no"].min() if metric_dd.value == "records" else sub["timestamp"].min(),
            x1=sub["rec_no"].max() if metric_dd.value == "records" else sub["timestamp"].max(),
            y0=outlier_threshold,
            y1=outlier_threshold,
            line=dict(
                color="red",
                width=2,
                dash="dot"  # 'dot' for dotted line
            )
        )

        attach_image_click(fig_widget_total_poro, sub)
        display(fig_widget_total_poro)

    #--------------Plot 11: Pore Count-----------------
    with plot_pore_count_out:
        plot_pore_count_out.clear_output()

        trace_pore_count = go.Scatter(
            x=sub["rec_no"] if metric_dd.value == "records" else sub["timestamp"].astype("O"),
            y=sub["numPores"],
            mode="markers",
            marker=dict(size=6, color="steelblue"),
            customdata=sub[["row_idx"]],
            hovertemplate=f"{x_label}=%{{x}}<br>Pore Count=%{{y}}<extra></extra>",
            name=""
        )

        fig_widget_pore_count = go.FigureWidget(data=[trace_pore_count])
        fig_widget_pore_count.update_layout(
            title=dict(text=f"{mc} {op_dd.value} Number of Pores Detected", x=0.5),
            xaxis_title="Record Number" if metric_dd.value == "records" else "Dates",
            yaxis_title="Pore Count",
            plot_bgcolor="white",
            paper_bgcolor="white",
            margin=dict(l=60, r=60, t=50, b=50),
            showlegend=False,
            xaxis=dict(showgrid=False,showline=True,linecolor="black",tickformat="%Y-%m-%d %H:%M" if metric_dd.value != "records" else None,tickmode="linear",dtick=50 if metric_dd.value == "records" else None),
            yaxis=dict(showgrid=False, showline=True, linecolor="black"),
            height=500,
            width=1100,
        )

        attach_image_click(fig_widget_pore_count, sub)
        display(fig_widget_pore_count)

    #-----------Stats-----------------
    with stats_out:
        stats_out.clear_output()
        rng = sub[metric].max() - sub[metric].min()
        std = sub[metric].std(ddof=0)
        tol = sub["tolerance"].dropna().iloc[0] if not sub["tolerance"].dropna().empty else np.nan
        span = sub["timestamp"].max() - sub["timestamp"].min()
        start_date = sub["timestamp"].min()
        end_date   = sub["timestamp"].max()
        flyer_max = int(list(max_outlier_counts.values())[0])
        flyer_sum = int(list(sum_outlier_counts.values())[0])
        display(Markdown(f"""
* **Side** : **Left**
* **Count**: {len(sub)}
* **Range**: {rng:.4g}
* **σ**: {std:.4g}
* **First Date**: {start_date}
* **Last Date**: {end_date}
* **Elapsed**: {span}
* **Flyer of Pores >1.6**: {flyer_max}
* **Flyer of total pores >6.0**: {flyer_sum}
"""))
        
    # ------------ view toggle ------------
    if view_dd.value == "coordinates":
        plot_out.layout.display          = ""      # visible
        plot_x_out.layout.display        = ""
        plot_y_out.layout.display        = ""
        plot_total_out.layout.display    = ""
        plot_y_vehicle_out.layout.display = "none"
        plot_z_vehicle_out.layout.display = "none"
        plot_status.layout.display = ""
        plot_len_out.layout.display = ""
        plot_max_poro_out.layout.display = ""
        plot_total_poro_out.layout.display = ""
        plot_pore_count_out.layout.display = ""
    else:
        plot_out.layout.display          = "none"
        plot_x_out.layout.display        = ""
        plot_y_out.layout.display        = "none"
        plot_total_out.layout.display    = ""
        plot_y_vehicle_out.layout.display = ""
        plot_z_vehicle_out.layout.display = ""
        plot_status.layout.display = ""
        plot_len_out.layout.display = ""
        plot_max_poro_out.layout.display = ""
        plot_total_poro_out.layout.display = ""
        plot_pore_count_out.layout.display = ""

    #-----gauge r ------
    # --------------------------
    # Input Data (example)
    # --------------------------
    # Assuming you already have master_df in your dashboard
    # Replace this with your actual DataFrame
    selected_columns = ['op_base', 'deviationSize', 'phy_x', 'phy_y', 'phy_z']
    gauge_df = master_df[selected_columns].copy()
    gauge_df = gauge_df.sort_values(by='op_base').reset_index(drop=True)

    def compute_result_df():
        rows = []

        # ----- Build main rows for each operation -----
        for op_base, group in gauge_df.groupby('op_base', sort=False):
            std_dev_deviation = group['deviationSize'].std()

            # Replicate rows
            for idx, (_, row) in enumerate(group.iterrows(), start=1):
                rows.append({
                    'Operation Name': op_base if idx == 1 else '',
                    'Replicate': idx,
                    'X': row['phy_x'],
                    'Y': row['phy_y'],
                    'Z': row['phy_z'],
                    'Total Deviation to CAD': row['deviationSize'],
                    'Standard Deviation of Deviation from CAD Measurements': std_dev_deviation if idx == 1 else ''
                })

            # Summary rows per operation
            avg = group[['deviationSize', 'phy_x', 'phy_y', 'phy_z']].mean()
            rng = group[['deviationSize', 'phy_x', 'phy_y', 'phy_z']].max() - group[['deviationSize', 'phy_x', 'phy_y', 'phy_z']].min()
            std = group[['deviationSize', 'phy_x', 'phy_y', 'phy_z']].std()

            for label, stats in zip(['Average', 'Range', 'Standard Deviation'], [avg, rng, std]):
                rows.append({
                    'Operation Name': '',
                    'Replicate': label,
                    'X': stats['phy_x'],
                    'Y': stats['phy_y'],
                    'Z': stats['phy_z'],
                    'Total Deviation to CAD': stats['deviationSize'],
                    'Standard Deviation of Deviation from CAD Measurements': ''
                })

        df = pd.DataFrame(rows)

        # ----- BUILD FINAL TOTAL SUMMARY -----
        range_rows = df[df["Replicate"] == "Range"]["Total Deviation to CAD"]

        avg_range = range_rows.mean()
        std_range = range_rows.std()
        d2_std = avg_range / 3.14   # Using d2 factor

        # EXACT FORMAT LIKE YOUR SCREENSHOT
        summary_rows = [
            ["Total", "Average Range", "", "", "", avg_range, ""],
            ["", "STD of the range", "", "", "", std_range, ""],
            ["", "Range using d2 factor from the table", "", "", "", d2_std, ""]
        ]

        df_summary = pd.DataFrame(summary_rows, columns=df.columns)

        final_df = pd.concat([df, df_summary], ignore_index=True)
        return final_df


    def save_to_excel(_):
        result_df = compute_result_df()
        output_path = "Test.xlsx"

        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            result_df.to_excel(writer, index=False, startrow=1, sheet_name='Sheet1')

            workbook = writer.book
            worksheet = writer.sheets['Sheet1']

            # Formats
            header_format = workbook.add_format({
                'bold': True, 'align': 'center', 'valign': 'vcenter',
                'bg_color': '#D9E1F2', 'border': 1
            })
            cell_format = workbook.add_format({'border': 1, 'num_format': '0.000'})
            bold_text = workbook.add_format({'bold': True, 'border': 1, 'num_format': '0.000'})
            merge_format = workbook.add_format({'border': 1, 'align': 'center', 'valign': 'vcenter', 'num_format': '0.000'})

            # Column widths
            col_widths = [18, 30, 12, 12, 12, 28, 55]
            for i, w in enumerate(col_widths):
                worksheet.set_column(i, i, w)

            # Headers
            worksheet.merge_range('A1:A2', 'Operation Name', header_format)
            worksheet.merge_range('B1:B2', 'Replicate', header_format)
            worksheet.merge_range('C1:E1', 'Measurements', header_format)
            worksheet.write('C2', 'X', header_format)
            worksheet.write('D2', 'Y', header_format)
            worksheet.write('E2', 'Z', header_format)
            worksheet.merge_range('F1:F2', 'Total Deviation to CAD', header_format)
            worksheet.merge_range('G1:G2', 'Standard Deviation of Deviation from CAD Measurements', header_format)

            # Write rows with formatting
            for row_idx in range(2, len(result_df) + 2):
                replicate_value = result_df.iloc[row_idx - 2]['Replicate']
                rep = str(replicate_value).lower()

                # Bold formats for summary rows
                if rep in ["average", "range", "standard deviation",
                        "average range", "std of the range", "range using d2 factor from the table"]:
                    fmt = bold_text
                else:
                    fmt = cell_format

                for col_idx in range(len(result_df.columns)):
                    val = result_df.iloc[row_idx - 2, col_idx]
                    worksheet.write(row_idx, col_idx, val, fmt)

            # ----- MERGE TOTAL SUMMARY BLOCK -----
            last_start = len(result_df) - 3 + 2   # +2 because of header rows
            last_end = len(result_df) - 1 + 2

            worksheet.merge_range(last_start, 0, last_end, 0, "Total", merge_format)

            print("Excel saved successfully as 'Test.xlsx'")

    save_button.on_click(save_to_excel)

In [22]:
def do_folder_load(_):
    """Load all JSON files from the selected folder into master_df."""
    global master_df, length_df, loaded_files
    folder = folder_chooser.selected_path
    if not folder or not os.path.isdir(folder):
        print("[INFO] Please select a valid folder.")
        return

    fresh = load_skillreal_folder(folder)
    new   = fresh[~fresh["file"].isin(loaded_files)]
    if new.empty:
        print("[INFO] No new JSON files found.")
        return

    master_df  = pd.concat([master_df, new], ignore_index=True)
    ts_min = master_df["timestamp"].min().date()
    ts_max = master_df["timestamp"].max().date()
    start_date_picker.value = ts_min
    end_date_picker.value   = ts_max
    length_df  = compute_feature_lengths(master_df)
    loaded_files.update(new["file"].unique())

    op_dd.options = sorted(master_df["feature"].unique())
    if op_dd.options:
        op_dd.value = op_dd.options[0]        # triggers update_dashboard via observer

def do_reset(_):
    """Clear all data and plots."""
    global master_df, length_df, loaded_files
    master_df = pd.DataFrame()
    length_df = pd.DataFrame()
    loaded_files.clear()
    op_dd.options = []
    for out in (plot_out, plot_total_out, plot_y_out, plot_x_out,
                plot_y_vehicle_out, plot_z_vehicle_out, plot_status,
                plot_len_out, plot_max_poro_out, plot_total_poro_out,
                stats_out):
        out.clear_output()

In [23]:
load_folder_btn.on_click(do_folder_load)
reset_btn.on_click(do_reset)

op_dd.observe(update_dashboard, names="value")
axis_dd.observe(update_dashboard, names="value")
scale_dd.observe(update_dashboard, names="value")
view_dd.observe(update_dashboard, names="value")
metric_dd.observe(update_dashboard, names="value")
start_date_picker.observe(update_dashboard, names="value")
end_date_picker.observe(update_dashboard, names="value")

In [24]:
dashboard = w.VBox([
    folder_chooser,
    w.HBox([load_folder_btn, reset_btn, save_button]),
    w.HBox([start_date_picker, end_date_picker]),
    w.HBox([op_dd, view_dd, metric_dd]),
    stats_out,
    plot_total_out,
    plot_x_out,
    plot_y_out,
    plot_out,
    plot_y_vehicle_out,
    plot_z_vehicle_out,
    plot_status,
    plot_len_out,
    plot_max_poro_out,
    plot_total_poro_out,
    plot_pore_count_out,
    image_out
])

display(dashboard)